In [5]:
from semanticscholar import SemanticScholar
import csv
import requests
import os
API_KEY = os.getenv('S2APIKEY')

In [6]:


import argparse
import os
from requests import Session
from typing import Generator, Union

import urllib3
urllib3.disable_warnings()


In [7]:

output_dir = "papers"
os.makedirs(output_dir, exist_ok=True)

download_errors = "ignore" 

extra_fields = ["keywords", "topics", "tldr", "citations"]

seen_paper_ids = set()

authors = [
    ("First Last1", ["John Doe", "Jane Smith"]),  # Replace with your list
    ("Second Last2", ["Another Author"]),
]

In [8]:
def download_paper(url, filename):
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(os.path.join(output_dir, filename), "wb") as f:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        f.write(chunk)
        else:
            print(f"PDF download failed for {url} ({response.status_code})")
    except Exception as e:
        if download_errors == "raise":
            raise e
        elif download_errors == "warn":
            print(f"Error downloading PDF: {e}")

In [9]:
def process_paper(author_name, paper_id, paper_details, seen_paper_ids, fields):
    """Processes a paper and writes data to a CSV file.

    Args:
        author_name (str): Name of the author.
        paper_id (str): ID of the paper.
        paper_details (dict): Dictionary containing paper details from the API.
        seen_paper_ids (set): Set of processed paper IDs to avoid duplicates.
        fields (list): List of fields to extract from the paper details.

    Returns:
        None
    """

    if paper_id in seen_paper_ids:
        return

    seen_paper_ids.add(paper_id)

    data = {}
    for field in fields:
        data[field] = paper_details.get(field)

    # Extract desired data:
    author_full_name = paper_details.get("authors", [{"name": author_name}])[0]["name"]
    title = paper_details.get("title")
    publication_date = paper_details.get("year")
    url = paper_details.get("url")
    abstract = paper_details.get("abstract")
    citation_count = paper_details.get("citationCount")
    influential_citation_count = paper_details.get("influentialCitationCount")
    open_access = paper_details.get("isOpenAccess")
    open_access_pdf = paper_details.get("openAccessPdf")
    fields_of_study = paper_details.get("fieldsOfStudy")
    s2_fields_of_study = paper_details.get("s2FieldsOfStudy")
    publication_types = paper_details.get("publicationTypes")
    journal = paper_details.get("journal")
    tldr = paper_details.get("tldr")

    # Prepare data for CSV:
    data["author_name"] = author_full_name
    data["paper_id"] = paper_id
    data["title"] = title
    data["hindex"] = data.get("hIndex")  # Assume this comes from author details
    data["semantic_scholar_url"] = url
    data["affiliations"] = ", ".join([affiliation["name"] for affiliation in data.get("affiliations", [])])
    data["citation_count"] = citation_count
    data["publication_types"] = ", ".join(publication_types)
    data["influential_citation_count"] = influential_citation_count
    data["publication_venue"] = journal.get("name")
    data["citation_count_of_the_paper"] = citation_count
    data["field_of_study"] = ", ".join(fields_of_study)
    data["s2_fields_of_study"] = ", ".join(s2_fields_of_study)
    data["journal"] = journal.get("name")
    data["abstract"] = abstract
    data["tldr"] = tldr
    data["publication_date"] = publication_date

    # Download PDF if open access and not already downloaded:
    if open_access and open_access_pdf:
        pdf_path = download_paper(paper_id, "papers")
        if pdf_path:
            data["pdf_path"] = pdf_path

    # Write data to CSV:
    with open("papers.csv", "a", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fields + ["author_name", "paper_id", "pdf_path"])
        writer.writerow(data)


In [10]:
def process_author(author_name, all_authors):
    try:
        # Use API key from environment variable
        sch = SemanticScholar(api_key=API_KEY)

        # Find author and retrieve ID
        author_profile = sch.search_author(query=author_name)
        author_id = author_profile[0].id

        # Get author publications
        author_papers = sch.get_author_papers(author_id)

        for paper in author_papers:
            try:
                paper_details = sch.paper_by_id(paper['paperId'])

                process_paper(author_name, all_authors, paper['paperId'], paper_details)
            except Exception as e:
                print(f"Error processing paper {paper['paperId']}: {e}")

    except Exception as e:
        if download_errors == "raise":
            raise e
        elif download_errors == "warn":
            print(f"Error processing author {author_name}: {e}")
        else:
            print(f"Author not found: {author_name}")

In [11]:
sch = SemanticScholar(api_key=API_KEY)

In [23]:
out = sch.search_author(query="Graham Neubig")
author_id = out[0].authorId
url = out[0].url
paper_count = out[0].paperCount
# write similar code for citationCount, hIndex

In [98]:
out[0].papers[0].keys()

dict_keys(['paperId', 'externalIds', 'corpusId', 'publicationVenue', 'url', 'title', 'abstract', 'venue', 'year', 'referenceCount', 'citationCount', 'influentialCitationCount', 'isOpenAccess', 'openAccessPdf', 'fieldsOfStudy', 's2FieldsOfStudy', 'publicationTypes', 'publicationDate', 'journal', 'authors'])

In [96]:
out[0].tldr

AttributeError: 'Author' object has no attribute 'tldr'

In [190]:
author_name = "Graham Neubig"
search_response = requests.get(
        'https://api.semanticscholar.org/graph/v1/author/search',
        headers=headers,
        params={'query': author_name}
    )
search_response

<Response [200]>

In [ ]:
out[0].papers[0].keys()

dict_keys(['paperId', 'externalIds', 'corpusId', 'publicationVenue', 'url', 'title', 'abstract', 'venue', 'year', 'referenceCount', 'citationCount', 'influentialCitationCount', 'isOpenAccess', 'openAccessPdf', 'fieldsOfStudy', 's2FieldsOfStudy', 'publicationTypes', 'publicationDate', 'journal', 'authors'])

In [177]:
a = 's'
a = a+'b'
a

'sb'

In [187]:
import requests
import json
import os

# Fetch API key from environment variables
api_key = API_KEY
if not api_key:
    raise EnvironmentError("S2_API_KEY environment variable not set.")

# Set up the headers with the API key
headers = {
    "x-api-key": api_key
}

# Specify the author_id and the fields you want to fetch
author_id = "1700325"
fields = "authorId,name,url,hIndex,affiliations,paperCount,citationCount"
fields +=",papers.paperId,papers.title,papers.year,papers.url,papers.abstract,papers.authors"
fields += ",papers.externalIds,papers.isOpenAccess,papers.openAccessPdf,papers.fieldsOfStudy"
fields += ",papers.influentialCitationCount,papers.journal"
# Make the GET request to fetch author details with the specified fields
response = requests.get(
    f'https://api.semanticscholar.org/graph/v1/author/{author_id}',
    headers=headers,
    params={'fields': fields}
)

# Check if the request was successful
if response.status_code == 200:
    # Create the directory for raw output if it doesn't exist
    os.makedirs('raw_output_semantic', exist_ok=True)
    
    # Define the path for the JSON file
    json_file_path = os.path.join(f'author_{author_id}_data.json')
    
    # Save the JSON response
    with open(json_file_path, 'w') as json_file:
        json.dump(response.json(), json_file, indent=2)
        print(f"Saved JSON response to {json_file_path}")
    
else:
    print(f"Error: {response.status_code}")
    if response.text:
        print(json.dumps(response.json(), indent=2))
    else:
        print("No additional error information is provided.")


Saved JSON response to author_1700325_data.json


In [191]:
import requests
import json
import os

def fetch_and_process_author_papers(author_name, year, save_raw=False):
    if not API_KEY:
        raise EnvironmentError("S2_API_KEY environment variable not set.")

    # Set up the headers with the API key
    headers = {
        "x-api-key": API_KEY
    }

    # First, find the author ID using the search endpoint
    search_response = requests.get(
        'https://api.semanticscholar.org/graph/v1/author/search',
        headers=headers,
        params={'query': author_name}
    )
    if search_response.status_code != 200:
        raise Exception(f"Error finding author ID: {search_response.status_code}")

    author_id = search_response.json()['data'][0]['authorId']

    # Specify the fields you want to fetch
    fields = "authorId,name,url,hIndex,affiliations,paperCount,citationCount"
    fields += ",papers.paperId,papers.title,papers.year,papers.url,papers.abstract,papers.authors"
    fields += ",papers.externalIds,papers.isOpenAccess,papers.openAccessPdf,papers.fieldsOfStudy"
    fields += ",papers.influentialCitationCount,papers.journal"

    # Make the GET request to fetch author details with the specified fields
    response = requests.get(
        f'https://api.semanticscholar.org/graph/v1/author/{author_id}',
        headers=headers,
        params={'fields': fields}
    )

    # Check if the request was successful
    if response.status_code == 200:
        author_data = response.json()

        # Define paths for the raw and processed data
        raw_data_dir = 'data/paper/raw_output'
        processed_data_dir = 'data/paper/processed_output'
        raw_file_name = f'raw_output_{author_name}_{author_id}.json'
        processed_file_name = f'filtered_output_{author_name}_{author_id}.json'

        # Optionally save the raw output
        if save_raw:
            os.makedirs(raw_data_dir, exist_ok=True)
            raw_json_file_path = os.path.join(raw_data_dir, raw_file_name)
            with open(raw_json_file_path, 'w') as raw_file:
                json.dump(author_data, raw_file, indent=2)
                print(f"Saved raw data to {raw_json_file_path}")

        # Process and filter the data
        filtered_papers = [
            paper for paper in author_data.get('papers', [])
            if str(paper.get('year')) == str(year) and paper.get('isOpenAccess')
        ]

        # Save the processed data
        os.makedirs(processed_data_dir, exist_ok=True)
        processed_json_file_path = os.path.join(processed_data_dir, processed_file_name)
        with open(processed_json_file_path, 'w') as processed_file:
            json.dump(filtered_papers, processed_file, indent=2)
            print(f"Saved processed data to {processed_json_file_path}")
    else:
        print(f"Error fetching author data: {response.status_code}")
        if response.text:
            print(json.dumps(response.json(), indent=2))
        else:
            print("No additional error information is provided.")

# Usage
# Replace 'Graham Neubig' with the actual author's name and '2023' with the target year
fetch_and_process_author_papers('Graham Neubig', 2023, save_raw=False)

Saved processed data to data/paper/processed_output/filtered_output_Graham Neubig_1700325.json


In [188]:
# Now, let's process the saved JSON data
def process_saved_data(json_file_path):
    # Create the directory for processed output if it doesn't exist
    processed_output_dir = 'processed_output_semantic'
    os.makedirs(processed_output_dir, exist_ok=True)
    
    # Load the JSON data for processing
    with open(json_file_path, 'r') as json_file:
        author_data = json.load(json_file)
        
        # Filter for open access papers from the year 2023
        filtered_papers = [
            paper for paper in author_data.get('papers', [])
            if (paper.get('year') == "2023")is True
        ]
        
        # Define the path for the processed JSON file
        processed_json_file_path = os.path.join(processed_output_dir, f'author_{author_id}_processed_data.json')
        
        # Save the filtered papers
        with open(processed_json_file_path, 'w') as processed_json_file:
            json.dump(filtered_papers, processed_json_file, indent=2)
            print(f"Saved processed data to {processed_json_file_path}")

# Call the processing function if the raw data was successfully saved
if response.status_code == 200:
    process_saved_data(json_file_path)

Saved processed data to processed_output_semantic/author_1700325_processed_data.json


In [185]:
with open(json_file_path, 'r') as json_file:
    author_data = json.load(json_file)

In [186]:
author_data.get('papers')[0].get('isOpenAccess')

False

In [161]:
filtered_papers = [
    paper for paper in author_data.get('papers', [])
    if str(paper.get('year')) == "2023" and paper.get('isOpenAccess')
]
len(filtered_papers)

27

In [159]:
for paper in author_data.get('papers'):
    if(paper.get('year')) ==2023 and paper.get('isOpenAccess') == True:
        print(paper.get('title'))

Cross-Modal Fine-Tuning: Align then Refine
ChatGPT MT: Competitive for High- (but Not Low-) Resource Languages
GlobalBench: A Benchmark for Global Progress in Natural Language Processing
Learning Performance-Improving Code Edits
CodeBERTScore: Evaluating Code Generation with Pretrained Models of Code
Neural Machine Translation for the Indigenous Languages of the Americas: An Introduction
User-Centric Evaluation of OCR Systems for Kwak’wala
Multi-Dimensional Evaluation of Text Summarization with In-Context Learning
A Gold Standard Dataset for the Reviewer Assignment Problem
SigMoreFun Submission to the SIGMORPHON Shared Task on Interlinear Glossing
Bridging the Gap: A Survey on Integrating (Human) Feedback for Natural Language Generation
FacTool: Factuality Detection in Generative AI - A Tool Augmented Framework for Multi-Task and Multi-Domain Scenarios
Active Retrieval Augmented Generation
Large Language Models Enable Few-Shot Clustering
Solving NLP Problems through Human-System Collab

In [164]:
import json
import os

# Assuming `response` is the response object from the requests call
if response.status_code == 200:
    author_data = response.json()

    # Directory where the raw and processed data will be saved
    raw_data_dir = 'raw_output_semantic'
    processed_data_dir = 'processed_output_semantic'

    # Create directories if they don't exist
    os.makedirs(raw_data_dir, exist_ok=True)
    os.makedirs(processed_data_dir, exist_ok=True)

    # Define the path for the raw JSON file and save the raw response
    raw_json_file_path = os.path.join(raw_data_dir, f'author_{author_id}_data.json')
    with open(raw_json_file_path, 'w') as raw_file:
        json.dump(author_data, raw_file, indent=2)
        print(f"Saved raw data to {raw_json_file_path}")
    
    # Now let's filter the papers
    filtered_papers = [
        paper for paper in author_data.get('papers', [])
        if str(paper.get('year')) == "2023" and paper.get('isOpenAccess')
    ]

    # Define the path for the processed JSON file and save the processed data
    processed_json_file_path = os.path.join(processed_data_dir, f'author_{author_id}_processed_data.json')
    with open(processed_json_file_path, 'w') as processed_file:
        json.dump(filtered_papers, processed_file, indent=2)
        print(f"Saved processed data to {processed_json_file_path}")

    # Print the number of filtered papers for verification
    print(f"Number of filtered papers: {len(filtered_papers)}")
else:
    print(f"Error: {response.status_code}")
    if response.text:
        print(json.dumps(response.json(), indent=2))
    else:
        print("No additional error information is provided.")

Saved raw data to raw_output_semantic/author_1700325_data.json
Saved processed data to processed_output_semantic/author_1700325_processed_data.json
Number of filtered papers: 27


In [ ]:
out[0].papers[0].keys()

dict_keys(['paperId', 'externalIds', 'corpusId', 'publicationVenue', 'url', 'title', 'abstract', 'venue', 'year', 'referenceCount', 'citationCount', 'influentialCitationCount', 'isOpenAccess', 'openAccessPdf', 'fieldsOfStudy', 's2FieldsOfStudy', 'publicationTypes', 'publicationDate', 'journal', 'authors'])

In [50]:
key = x[1]
value = out[0].__getattribute__(key)
value

{'DBLP': ['Graham Neubig']}

In [42]:
for i in raout[0].keys():
    print(list(out[0].keys())[i])
    print( out[0].list(out[0].keys())[i].keys() )

TypeError: 'dict_keys' object cannot be interpreted as an integer

In [21]:
file = open("log.txt", "w")
file.write(str(out[0]) )

1232796

In [75]:
# out = sch.search_author(query="Graham Neubig")
# then out[0] is a dict with keys dict_keys(['authorId', 'externalIds', 'url', 'name', 'affiliations', 'homepage', 'paperCount', 'citationCount', 'hIndex', 'papers'])
#out[0].papers[0] has dict_keys(['paperId', 'externalIds', 'corpusId', 'publicationVenue', 'url', 'title', 'abstract', 'venue', 'year', 'referenceCount', 'citationCount', 'influentialCitationCount', 'isOpenAccess', 'openAccessPdf', 'fieldsOfStudy', 's2FieldsOfStudy', 'publicationTypes', 'publicationDate', 'journal', 'authors'])

# out[0].papers[0].externalIds.keys() has dict_keys(['ArXiv', 'DBLP', 'DOI', 'CorpusId'])
# out[0].papers[0].publicationVenue.keys()dict_keys(['id', 'name', 'alternate_names', 'issn', 'url'])
# out[0].papers[0].journal.keys() dict_keys(['volume', 'name'])
# out[0].papers[0].authors[0] is a list of dicts 
# [{'authorId': '2279677197', 'name': 'Abhika Mishra'}, {'authorId': '35584853', 'name': 'Akari Asai'}, {'authorId': '143820870', 'name': 'Vidhisha Balachandran'}]
# if AttributeError for any of the out.atrributes then just skip that in adding 

year
